<a href="https://colab.research.google.com/github/Chandradithya8/Advanced_Regression/blob/main/Advanced_regression2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score

Feature engineering for train dataset

In [2]:
df_train=pd.read_csv("/content/train_ad.csv")
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
## splitting the categorical variables
categories=[features for features in df_train.columns if df_train[features].dtypes=='O']
print(len(categories))
print(categories)

43
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [4]:
## categorical variables with nan values
categories_na=[features for features in categories if  df_train[features].isnull().sum()>0]
len(categories_na)

16

In [5]:
## replacing nan values with new features
for features in categories_na:
  df_train[features]=np.where(df_train[features].isnull(),'Missing',df_train[features])
df_train[categories_na].isnull().sum()  

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [6]:
## splitting the numerical variables
numerical=[features for features in df_train.columns if features not in categories]
len(numerical)

38

In [7]:
## splitting the numerical variables with nan values
numerical_na=[features for features in numerical if df_train[features].isnull().sum()>0]
len(numerical_na)

3

In [8]:
## replacing nan values with the median
for features in numerical_na:
  df_train[features]=np.where(df_train[features].isnull(),df_train[features].median(),df_train[features])
df_train[numerical_na].isnull().sum()  

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [9]:
df_train.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [10]:
## splitting the temporal variables(date time variables)
temporal=[features for features in numerical if 'Yr' in features or 'Year' in features]
temporal

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [11]:
for features in temporal:
  if 'YrSold' not in features:
    df_train[features]=df_train['YrSold']-df_train[features]
df_train[numerical].head()    

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,5,5,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,5.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,31,31,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,31.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,7,6,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,7.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,91,36,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,8.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,8,8,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,8.0,3,836,192,84,0,0,0,0,0,12,2008,250000


In [12]:
for features in ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']:
  if 0 in df_train[features].unique():
    pass
  else:
    df_train[features]=np.log(df_train[features])
df_train[numerical].head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,4.174387,9.041922,7,5,5,5,196.0,706,0,150,856,6.752270,854,0,7.444249,1,0,2,1,3,1,8,0,5.0,2,548,0,61,0,0,0,0,0,2,2008,12.247694
1,2,20,4.382027,9.169518,6,8,31,31,0.0,978,0,284,1262,7.140453,0,0,7.140453,0,1,2,0,3,1,6,1,31.0,2,460,298,0,0,0,0,0,0,5,2007,12.109011
2,3,60,4.219508,9.328123,7,5,7,6,162.0,486,0,434,920,6.824374,866,0,7.487734,1,0,2,1,3,1,6,1,7.0,2,608,0,42,0,0,0,0,0,9,2008,12.317167
3,4,70,4.094345,9.164296,7,5,91,36,0.0,216,0,540,756,6.867974,756,0,7.448334,1,0,1,0,3,1,7,1,8.0,3,642,0,35,272,0,0,0,0,2,2006,11.849398
4,5,60,4.430817,9.565214,8,5,8,8,350.0,655,0,490,1145,7.043160,1053,0,7.695303,1,0,2,1,4,1,9,1,8.0,3,836,192,84,0,0,0,0,0,12,2008,12.429216


In [13]:
## handling the categorical values
for features in categories:
  data_features=df_train.groupby(features)['SalePrice'].mean().sort_values().index
  labels_ordered={i:k for k,i in enumerate(data_features,0)}
  df_train[features]=df_train[features].map(labels_ordered)

In [14]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,4.174387,9.041922,1,2,0,1,1,0,0,15,3,4,3,6,7,5,5,5,1,2,12,13,2,196.0,2,3,5,3,3,1,6,706,5,0,150,856,5,...,1,5,6.752270,854,0,7.444249,1,0,2,1,3,1,2,8,6,0,1,5,5.0,2,2,548,3,5,2,0,61,0,0,0,0,0,4,2,0,2,2008,4,4,12.247694
1,2,20,3,4.382027,9.169518,1,2,0,1,1,2,0,20,1,4,3,4,6,8,31,31,1,2,6,6,1,0.0,1,3,2,3,3,4,4,978,5,0,284,1262,5,...,1,5,7.140453,0,0,7.140453,0,1,2,0,3,1,1,6,6,1,3,5,31.0,2,2,460,3,5,2,298,0,0,0,0,0,0,4,2,0,5,2007,4,4,12.109011
2,3,60,3,4.219508,9.328123,1,2,1,1,1,0,0,15,3,4,3,6,7,5,7,6,1,2,12,13,2,162.0,2,3,5,3,3,2,6,486,5,0,434,920,5,...,1,5,6.824374,866,0,7.487734,1,0,2,1,3,1,2,6,6,1,3,5,7.0,2,2,608,3,5,2,0,42,0,0,0,0,0,4,2,0,9,2008,4,4,12.317167
3,4,70,3,4.094345,9.164296,1,2,1,1,1,1,0,17,3,4,3,6,7,5,91,36,1,2,4,7,1,0.0,1,3,1,2,4,1,4,216,5,0,540,756,5,...,1,5,6.867974,756,0,7.448334,1,0,1,0,3,1,2,7,6,1,4,2,8.0,1,3,642,3,5,2,0,35,272,0,0,0,0,4,2,0,2,2006,4,1,11.849398
4,5,60,3,4.430817,9.565214,1,2,1,1,1,2,0,24,3,4,3,6,8,5,8,8,1,2,12,13,2,350.0,2,3,5,3,3,3,6,655,5,0,490,1145,5,...,1,5,7.043160,1053,0,7.695303,1,0,2,1,4,1,2,9,6,1,3,5,8.0,2,3,836,3,5,2,192,84,0,0,0,0,0,4,2,0,12,2008,4,4,12.429216


In [15]:
x_train=df_train.drop(['SalePrice','Id'],axis=1)
y_train=df_train['SalePrice']

In [16]:
## feature scaling-StandardScaler
scaler=StandardScaler()
scaling=scaler.fit_transform(x_train)
scaling

array([[ 0.07337496,  0.35904396, -0.07586857, ...,  0.13877749,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.57242366, ..., -0.61443862,
        -0.22029532,  0.18996938],
       [ 0.07337496,  0.35904396,  0.06500658, ...,  0.13877749,
        -0.22029532,  0.18996938],
       ...,
       [ 0.30985939,  0.35904396, -0.02820043, ...,  1.64520971,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.06500658, ...,  1.64520971,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.3709213 , ...,  0.13877749,
        -0.22029532,  0.18996938]])

In [17]:
x_train=pd.DataFrame(scaling,columns=x_train.columns)
x_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,0.359044,-0.075869,-0.133231,0.064238,0.244717,-0.657040,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-1.043259,-0.869941,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.514104,1.052302,0.339798,1.089523,0.618960,0.094944,-0.590555,1.006001,0.575425,0.337745,-0.288653,-0.944591,-0.459303,0.120988,0.891179,0.263813,0.297699,-0.803570,1.161852,-0.120242,0.529260,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,0.912210,0.237384,-0.951226,-0.944190,0.627813,-1.008328,0.318475,0.311725,0.351000,0.236233,0.303263,0.289745,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,0.138777,-0.220295,0.189969
1,-0.872563,0.359044,0.572424,0.113442,0.064238,0.244717,-0.657040,-0.111168,0.02618,1.418260,-0.225716,1.270347,-1.981037,0.051631,0.12422,-0.238178,-0.071836,2.179628,-0.183465,0.390141,-0.502859,-0.102123,-0.858917,-0.924976,-0.634468,-0.570750,-0.689604,0.339798,-0.731199,0.618960,0.094944,2.220999,-0.136559,1.171992,0.337745,-0.288653,-0.641228,0.466465,0.120988,0.891179,0.263813,0.297699,0.418585,-0.795163,-0.120242,-0.381846,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.771091,-0.318683,0.237384,0.600495,0.526229,0.627813,0.073805,0.318475,0.311725,-0.060731,0.236233,0.303263,0.289745,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-0.489110,-0.614439,-0.220295,0.189969
2,0.073375,0.359044,0.065007,0.420061,0.064238,0.244717,0.872909,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-0.977121,-0.821476,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.325915,1.052302,0.339798,1.089523,0.618960,0.094944,0.346630,1.006001,0.092907,0.337745,-0.288653,-0.301643,-0.313369,0.120988,0.891179,0.263813,0.297699,-0.576560,1.189351,-0.120242,0.659675,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,-0.318683,0.237384,0.600495,0.526229,0.627813,-0.925087,0.318475,0.311725,0.631726,0.236233,0.303263,0.289745,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,0.990891,0.138777,-0.220295,0.189969
3,0.309859,0.359044,-0.325778,0.103347,0.064238,0.244717,0.872909,-0.111168,0.02618,0.466233,-0.225716,0.802144,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,1.800676,0.632464,-0.502859,-0.102123,-1.506654,-0.642215,-0.634468,-0.570750,-0.689604,0.339798,-1.338106,-0.655627,2.405256,-0.590555,-0.136559,-0.499274,0.337745,-0.288653,-0.061670,-0.687324,0.120988,-0.151386,0.263813,0.297699,-0.439287,0.937276,-0.120242,0.541511,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.735994,0.296763,0.237384,0.600495,1.261438,-1.152006,-0.883467,-0.801942,1.650307,0.790804,0.236233,0.303263,0.289745,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,-1.367655,-0.220295,-3.192412
4,0.073375,0.359044,0.724756,0.878409,0.064238,0.244717,0.872909,-0.111168,0.02618,1.418260,-0.225716,1.894617,0.111810,0.051631,0.12422,1.172531,1.374795,-0.517200,-0.944052,-0.

In [18]:
## handling the outliers
for features in x_train.columns:
  mean=x_train[features].mean()
  std=x_train[features].std()
  upper_bound=mean+(3*std)
  lower_bound=mean-(3*std)
  x_train[features]=np.where(x_train[features]>upper_bound,upper_bound,x_train[features])
  x_train[features]=np.where(x_train[features]<lower_bound,lower_bound,x_train[features])

In [19]:
x_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,0.359044,-0.075869,-0.133231,0.064238,0.244717,-0.657040,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-1.043259,-0.869941,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.514104,1.052302,0.339798,1.089523,0.618960,0.094944,-0.590555,1.006001,0.575425,0.337745,-0.288653,-0.944591,-0.459303,0.120988,0.891179,0.263813,0.297699,-0.803570,1.161852,-0.120242,0.529260,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,0.912210,0.237384,-0.951226,-0.944190,0.627813,-1.008328,0.318475,0.311725,0.351000,0.236233,0.303263,0.289745,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,0.138777,-0.220295,0.189969
1,-0.872563,0.359044,0.572424,0.113442,0.064238,0.244717,-0.657040,-0.111168,0.02618,1.418260,-0.225716,1.270347,-1.981037,0.051631,0.12422,-0.238178,-0.071836,2.179628,-0.183465,0.390141,-0.502859,-0.102123,-0.858917,-0.924976,-0.634468,-0.570750,-0.689604,0.339798,-0.731199,0.618960,0.094944,2.220999,-0.136559,1.171992,0.337745,-0.288653,-0.641228,0.466465,0.120988,0.891179,0.263813,0.297699,0.418585,-0.795163,-0.120242,-0.381846,-0.819964,3.001028,0.789741,-0.761621,0.163779,-0.211454,-0.771091,-0.318683,0.237384,0.600495,0.526229,0.627813,0.073805,0.318475,0.311725,-0.060731,0.236233,0.303263,0.289745,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-0.489110,-0.614439,-0.220295,0.189969
2,0.073375,0.359044,0.065007,0.420061,0.064238,0.244717,0.872909,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-0.977121,-0.821476,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.325915,1.052302,0.339798,1.089523,0.618960,0.094944,0.346630,1.006001,0.092907,0.337745,-0.288653,-0.301643,-0.313369,0.120988,0.891179,0.263813,0.297699,-0.576560,1.189351,-0.120242,0.659675,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,-0.318683,0.237384,0.600495,0.526229,0.627813,-0.925087,0.318475,0.311725,0.631726,0.236233,0.303263,0.289745,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,0.990891,0.138777,-0.220295,0.189969
3,0.309859,0.359044,-0.325778,0.103347,0.064238,0.244717,0.872909,-0.111168,0.02618,0.466233,-0.225716,0.802144,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,1.800676,0.632464,-0.502859,-0.102123,-1.506654,-0.642215,-0.634468,-0.570750,-0.689604,0.339798,-1.338106,-0.655627,2.405256,-0.590555,-0.136559,-0.499274,0.337745,-0.288653,-0.061670,-0.687324,0.120988,-0.151386,0.263813,0.297699,-0.439287,0.937276,-0.120242,0.541511,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.735994,0.296763,0.237384,0.600495,1.261438,-1.152006,-0.883467,-0.801942,1.650307,0.790804,0.236233,0.303263,0.289745,-0.752176,-0.176048,3.001028,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,-1.367655,-0.220295,-3.001028
4,0.073375,0.359044,0.724756,0.878409,0.064238,0.244717,0.872909,-0.111168,0.02618,1.418260,-0.225716,1.894617,0.111810,0.051631,0.12422,1.172531,1.374795,-0.517200,-0.944052,-0.

Feature Engineering for test dataset

In [20]:
df_test=pd.read_csv("/content/train_ad.csv")
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [21]:
## splitting the categorical data
categories=[features for features in df_test.columns if df_test[features].dtypes=='O']
len(categories)

43

In [22]:
## splitting the categorical data having nan values
categories_na=[features for features in categories if df_test[features].isnull().sum()>0]
len(categories_na)

16

In [23]:
## replacing nan values with new values
for features in categories_na:
  df_test[features]=np.where(df_test[features].isnull(),'Missing',df_test[features])
df_test[categories_na].isnull().sum()  

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [24]:
## splitting the data with numerical values
numerical=[features for features in df_test.columns if features not in categories]
len(numerical)

38

In [25]:
## splitting the  numerical data with nan values
numerical_na=[features for features in numerical if df_test[features].isnull().sum()>0]
len(numerical_na)

3

In [26]:
## replacing the nan values with median value
for features in numerical_na:
  df_test[features]=np.where(df_test[features].isnull(),df_test[features].median(),df_test[features])
df_test[numerical_na].isnull().sum()  

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [27]:
## splitting the temporal data
temporal=[features for features in numerical if 'Yr' in features or 'Year' in features]
temporal

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [28]:
for features in temporal:
  if 'YrSold' not in features:
    df_test[features]=df_test['YrSold']-df_test[features]
df_test.head()   

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000


In [29]:
for features in ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']:
  if 0 in df_test[features].unique():
    pass
  else:
    df_test[features]=np.log(df_test[features])
df_test[numerical].head()    

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,4.174387,9.041922,7,5,5,5,196.0,706,0,150,856,6.752270,854,0,7.444249,1,0,2,1,3,1,8,0,5.0,2,548,0,61,0,0,0,0,0,2,2008,12.247694
1,2,20,4.382027,9.169518,6,8,31,31,0.0,978,0,284,1262,7.140453,0,0,7.140453,0,1,2,0,3,1,6,1,31.0,2,460,298,0,0,0,0,0,0,5,2007,12.109011
2,3,60,4.219508,9.328123,7,5,7,6,162.0,486,0,434,920,6.824374,866,0,7.487734,1,0,2,1,3,1,6,1,7.0,2,608,0,42,0,0,0,0,0,9,2008,12.317167
3,4,70,4.094345,9.164296,7,5,91,36,0.0,216,0,540,756,6.867974,756,0,7.448334,1,0,1,0,3,1,7,1,8.0,3,642,0,35,272,0,0,0,0,2,2006,11.849398
4,5,60,4.430817,9.565214,8,5,8,8,350.0,655,0,490,1145,7.043160,1053,0,7.695303,1,0,2,1,4,1,9,1,8.0,3,836,192,84,0,0,0,0,0,12,2008,12.429216


In [30]:
## handling the categorical values
for features in categories:
  data_features=df_test.groupby(features)['SalePrice'].mean().sort_values().index
  dictionary={k:i for i,k in enumerate(data_features,0)}
  df_test[features]=df_test[features].map(dictionary)

In [31]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,4.174387,9.041922,1,2,0,1,1,0,0,15,3,4,3,6,7,5,5,5,1,2,12,13,2,196.0,2,3,5,3,3,1,6,706,5,0,150,856,5,...,1,5,6.752270,854,0,7.444249,1,0,2,1,3,1,2,8,6,0,1,5,5.0,2,2,548,3,5,2,0,61,0,0,0,0,0,4,2,0,2,2008,4,4,12.247694
1,2,20,3,4.382027,9.169518,1,2,0,1,1,2,0,20,1,4,3,4,6,8,31,31,1,2,6,6,1,0.0,1,3,2,3,3,4,4,978,5,0,284,1262,5,...,1,5,7.140453,0,0,7.140453,0,1,2,0,3,1,1,6,6,1,3,5,31.0,2,2,460,3,5,2,298,0,0,0,0,0,0,4,2,0,5,2007,4,4,12.109011
2,3,60,3,4.219508,9.328123,1,2,1,1,1,0,0,15,3,4,3,6,7,5,7,6,1,2,12,13,2,162.0,2,3,5,3,3,2,6,486,5,0,434,920,5,...,1,5,6.824374,866,0,7.487734,1,0,2,1,3,1,2,6,6,1,3,5,7.0,2,2,608,3,5,2,0,42,0,0,0,0,0,4,2,0,9,2008,4,4,12.317167
3,4,70,3,4.094345,9.164296,1,2,1,1,1,1,0,17,3,4,3,6,7,5,91,36,1,2,4,7,1,0.0,1,3,1,2,4,1,4,216,5,0,540,756,5,...,1,5,6.867974,756,0,7.448334,1,0,1,0,3,1,2,7,6,1,4,2,8.0,1,3,642,3,5,2,0,35,272,0,0,0,0,4,2,0,2,2006,4,1,11.849398
4,5,60,3,4.430817,9.565214,1,2,1,1,1,2,0,24,3,4,3,6,8,5,8,8,1,2,12,13,2,350.0,2,3,5,3,3,3,6,655,5,0,490,1145,5,...,1,5,7.043160,1053,0,7.695303,1,0,2,1,4,1,2,9,6,1,3,5,8.0,2,3,836,3,5,2,192,84,0,0,0,0,0,4,2,0,12,2008,4,4,12.429216


In [32]:
x=df_test.drop(['SalePrice','Id'],axis=1)
y=df_test['SalePrice']

In [33]:
## feature scaling
scaler=StandardScaler()
scaling=scaler.fit_transform(x)
scaling

array([[ 0.07337496,  0.35904396, -0.07586857, ...,  0.13877749,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.57242366, ..., -0.61443862,
        -0.22029532,  0.18996938],
       [ 0.07337496,  0.35904396,  0.06500658, ...,  0.13877749,
        -0.22029532,  0.18996938],
       ...,
       [ 0.30985939,  0.35904396, -0.02820043, ...,  1.64520971,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.06500658, ...,  1.64520971,
        -0.22029532,  0.18996938],
       [-0.87256276,  0.35904396,  0.3709213 , ...,  0.13877749,
        -0.22029532,  0.18996938]])

In [34]:
x=pd.DataFrame(scaling,columns=x.columns)
x.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,0.359044,-0.075869,-0.133231,0.064238,0.244717,-0.657040,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-1.043259,-0.869941,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.514104,1.052302,0.339798,1.089523,0.618960,0.094944,-0.590555,1.006001,0.575425,0.337745,-0.288653,-0.944591,-0.459303,0.120988,0.891179,0.263813,0.297699,-0.803570,1.161852,-0.120242,0.529260,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,0.912210,0.237384,-0.951226,-0.944190,0.627813,-1.008328,0.318475,0.311725,0.351000,0.236233,0.303263,0.289745,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,0.138777,-0.220295,0.189969
1,-0.872563,0.359044,0.572424,0.113442,0.064238,0.244717,-0.657040,-0.111168,0.02618,1.418260,-0.225716,1.270347,-1.981037,0.051631,0.12422,-0.238178,-0.071836,2.179628,-0.183465,0.390141,-0.502859,-0.102123,-0.858917,-0.924976,-0.634468,-0.570750,-0.689604,0.339798,-0.731199,0.618960,0.094944,2.220999,-0.136559,1.171992,0.337745,-0.288653,-0.641228,0.466465,0.120988,0.891179,0.263813,0.297699,0.418585,-0.795163,-0.120242,-0.381846,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.771091,-0.318683,0.237384,0.600495,0.526229,0.627813,0.073805,0.318475,0.311725,-0.060731,0.236233,0.303263,0.289745,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-0.489110,-0.614439,-0.220295,0.189969
2,0.073375,0.359044,0.065007,0.420061,0.064238,0.244717,0.872909,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-0.977121,-0.821476,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.325915,1.052302,0.339798,1.089523,0.618960,0.094944,0.346630,1.006001,0.092907,0.337745,-0.288653,-0.301643,-0.313369,0.120988,0.891179,0.263813,0.297699,-0.576560,1.189351,-0.120242,0.659675,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,-0.318683,0.237384,0.600495,0.526229,0.627813,-0.925087,0.318475,0.311725,0.631726,0.236233,0.303263,0.289745,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,0.990891,0.138777,-0.220295,0.189969
3,0.309859,0.359044,-0.325778,0.103347,0.064238,0.244717,0.872909,-0.111168,0.02618,0.466233,-0.225716,0.802144,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,1.800676,0.632464,-0.502859,-0.102123,-1.506654,-0.642215,-0.634468,-0.570750,-0.689604,0.339798,-1.338106,-0.655627,2.405256,-0.590555,-0.136559,-0.499274,0.337745,-0.288653,-0.061670,-0.687324,0.120988,-0.151386,0.263813,0.297699,-0.439287,0.937276,-0.120242,0.541511,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.735994,0.296763,0.237384,0.600495,1.261438,-1.152006,-0.883467,-0.801942,1.650307,0.790804,0.236233,0.303263,0.289745,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,-1.367655,-0.220295,-3.192412
4,0.073375,0.359044,0.724756,0.878409,0.064238,0.244717,0.872909,-0.111168,0.02618,1.418260,-0.225716,1.894617,0.111810,0.051631,0.12422,1.172531,1.374795,-0.517200,-0.944052,-0.

In [35]:
## handling the outliers
for features in x.columns:
  mean=x[features].mean()
  std=x[features].std()
  upper_bound=mean+(3*std)
  lower_bound=mean-(3*std)
  x[features]=np.where(x[features]>upper_bound,upper_bound,x[features])
  x[features]=np.where(x[features]<lower_bound,lower_bound,x[features])

In [36]:
x.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,0.359044,-0.075869,-0.133231,0.064238,0.244717,-0.657040,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-1.043259,-0.869941,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.514104,1.052302,0.339798,1.089523,0.618960,0.094944,-0.590555,1.006001,0.575425,0.337745,-0.288653,-0.944591,-0.459303,0.120988,0.891179,0.263813,0.297699,-0.803570,1.161852,-0.120242,0.529260,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,0.912210,0.237384,-0.951226,-0.944190,0.627813,-1.008328,0.318475,0.311725,0.351000,0.236233,0.303263,0.289745,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,0.138777,-0.220295,0.189969
1,-0.872563,0.359044,0.572424,0.113442,0.064238,0.244717,-0.657040,-0.111168,0.02618,1.418260,-0.225716,1.270347,-1.981037,0.051631,0.12422,-0.238178,-0.071836,2.179628,-0.183465,0.390141,-0.502859,-0.102123,-0.858917,-0.924976,-0.634468,-0.570750,-0.689604,0.339798,-0.731199,0.618960,0.094944,2.220999,-0.136559,1.171992,0.337745,-0.288653,-0.641228,0.466465,0.120988,0.891179,0.263813,0.297699,0.418585,-0.795163,-0.120242,-0.381846,-0.819964,3.001028,0.789741,-0.761621,0.163779,-0.211454,-0.771091,-0.318683,0.237384,0.600495,0.526229,0.627813,0.073805,0.318475,0.311725,-0.060731,0.236233,0.303263,0.289745,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-0.489110,-0.614439,-0.220295,0.189969
2,0.073375,0.359044,0.065007,0.420061,0.064238,0.244717,0.872909,-0.111168,0.02618,-0.485795,-0.225716,0.490009,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,-0.977121,-0.821476,-0.502859,-0.102123,1.084294,1.054349,0.481584,0.325915,1.052302,0.339798,1.089523,0.618960,0.094944,0.346630,1.006001,0.092907,0.337745,-0.288653,-0.301643,-0.313369,0.120988,0.891179,0.263813,0.297699,-0.576560,1.189351,-0.120242,0.659675,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.735994,-0.318683,0.237384,0.600495,0.526229,0.627813,-0.925087,0.318475,0.311725,0.631726,0.236233,0.303263,0.289745,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,0.990891,0.138777,-0.220295,0.189969
3,0.309859,0.359044,-0.325778,0.103347,0.064238,0.244717,0.872909,-0.111168,0.02618,0.466233,-0.225716,0.802144,0.111810,0.051631,0.12422,1.172531,0.651479,-0.517200,1.800676,0.632464,-0.502859,-0.102123,-1.506654,-0.642215,-0.634468,-0.570750,-0.689604,0.339798,-1.338106,-0.655627,2.405256,-0.590555,-0.136559,-0.499274,0.337745,-0.288653,-0.061670,-0.687324,0.120988,-0.151386,0.263813,0.297699,-0.439287,0.937276,-0.120242,0.541511,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.735994,0.296763,0.237384,0.600495,1.261438,-1.152006,-0.883467,-0.801942,1.650307,0.790804,0.236233,0.303263,0.289745,-0.752176,-0.176048,3.001028,-0.116339,-0.270208,-0.068692,-0.063305,0.437409,0.163717,-0.087688,-1.599111,-1.367655,-0.220295,-3.001028
4,0.073375,0.359044,0.724756,0.878409,0.064238,0.244717,0.872909,-0.111168,0.02618,1.418260,-0.225716,1.894617,0.111810,0.051631,0.12422,1.172531,1.374795,-0.517200,-0.944052,-0.

In [37]:
x.shape

(1460, 79)

In [38]:
## the greater the alpha value the less number of features will be selected
selected=SelectFromModel(Lasso(alpha=0.005,random_state=0))
selected.fit(x_train,y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, copy_X=True, fit_intercept=True,
                                max_iter=1000, normalize=False, positive=False,
                                precompute=False, random_state=0,
                                selection='cyclic', tol=0.0001,
                                warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [39]:
selected.get_support()

array([ True,  True, False,  True, False, False, False,  True, False,
        True, False,  True,  True, False, False, False,  True,  True,
        True,  True, False, False,  True, False, False, False,  True,
        True,  True,  True, False,  True, False,  True, False, False,
       False,  True, False,  True,  True, False,  True, False, False,
        True,  True, False,  True,  True, False,  True,  True, False,
        True,  True,  True, False, False,  True,  True,  True,  True,
       False,  True,  True, False, False, False,  True, False, False,
       False, False, False, False, False,  True,  True])

In [42]:
selected_features=x_train.columns[selected.get_support()]
selected_features

Index(['MSSubClass', 'MSZoning', 'LotArea', 'LandContour', 'LotConfig',
       'Neighborhood', 'Condition1', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'Exterior1st', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtExposure', 'BsmtFinSF1', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath',
       'HalfBath', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'PavedDrive', 'WoodDeckSF', 'ScreenPorch', 'SaleType', 'SaleCondition'],
      dtype='object')

In [45]:
x_train=x_train[selected_features]
x=x[selected_features]
x_train.shape

(1460, 40)

In [41]:
model1=LinearRegression()
model2=RandomForestRegressor()
model3=XGBRegressor()

In [47]:
train1=model1.fit(x_train,y_train)
train1

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
y_pred=train1.predict(x)
print(r2_score(y,y_pred))

0.9131817966777542


In [54]:
train2=model2.fit(x_train,y_train)
train2

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [55]:
y_pred=train2.predict(x)
print(r2_score(y,y_pred))

0.9844191179683168


In [56]:
train3=model3.fit(x_train,y_train)
train3

[05:50:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [57]:
y_pred=train3.predict(x)
print(r2_score(y,y_pred))

0.9516908220314269


From the above models RandomForestRegressor gives the highest accuracy of 98% . LinearRegression gives a accuracy of 91% and xgboostRegressor gives a accuracy of 95%. 